In [11]:
import os
import numpy as np
import pandas as pd
import mne 


# fifFileMddPath = "/Users/wachiii/Workschii/brain-mdd/data/balanced16channels/mdd1624.fif"
# fifFileHcPath = "/Users/wachiii/Workschii/brain-mdd/data/balanced16channels/hc1624.fif"

fifFileHcEpochPath = "/Users/wachiii/Workschii/brain-mdd/data/balanced16preprocess/hc1624preprocessed.fif"
fifFileMddEpochPath = "/Users/wachiii/Workschii/brain-mdd/data/balanced16preprocess/mdd1624preprocessed.fif"   

allFeatureDirPath = "/Users/wachiii/Workschii/brain-mdd/data/balancedfeatures"
# mddRaw = mne.io.read_raw_fif(fifFileMddPath)
# hcRaw = mne.io.read_raw_fif(fifFileHcPath)
# print(mddRaw)
# print(hcRaw)

## Preprocessing
- bandpass filter 0.5 - 40 Hz
- Epoching for 10 sec => 10*250 = 2500 samples

In [ ]:
# do preprocessing: bandpass filter 0.5 - 40 Hz
mddEeg = mddRaw.load_data()
hcEeg = hcRaw.load_data()

mddEeg.filter(0.5, 40, fir_design='firwin')
hcEeg.filter(0.5, 40, fir_design='firwin')

# plot psd only 1 channel
mddEeg.plot_psd(fmin=0.5, fmax=40, average=True, spatial_colors=True) 
hcEeg.plot_psd(fmin=0.5, fmax=40, average=True, spatial_colors=True)

# do epoching; fs = 250, total samples = 1784136 where 1784136/24 = 74339.0 samples per subject -> i want epoch for 10 seconds
# 10 seconds = 2500 samples

mddEpochs = mne.make_fixed_length_epochs(mddEeg, duration=10, preload=True)
hcEpochs = mne.make_fixed_length_epochs(hcEeg, duration=10, preload=True)

print(mddEpochs)
print(hcEpochs)

# mddEpochs.save("/Users/wachiii/Workschii/brain-mdd/data/balanced16preprocess/mdd1624preprocessed.fif", overwrite=True)
# hcEpochs.save("/Users/wachiii/Workschii/brain-mdd/data/balanced16preprocess/hc1624preprocessed.fif", overwrite=True)

In [ ]:
74339.0/(10*250)

## Feature Extraction

### 1. PSD

In [ ]:
import dotenv
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
from brainmdd.features.psd import EEGPowerSpectrum
import mne
import enum

dotenv.load_dotenv()

class ChannelMapping(enum.Enum):
    Fp1 = 22
    Fp2 = 9
    F3 = 24
    F4 = 124
    C3 = 36
    C4 = 104
    P3 = 52
    P4 = 92
    O1 = 70
    O2 = 83
    F7 = 33
    F8 = 122
    T3 = 45
    T4 = 108
    T5 = 58
    T6 = 96

# Declare Global Variables
NUM_CHANNELS = 16
MODMA_SFREQ = 250


# Declare Feature Dictionary
featureDict = {
    "norm_power": {
        "mdd": {
            "delta": [],
            "theta": [],
            "alpha": [],
            "beta": [],
            "gamma": []
            },
        "control": {
            "delta": [],
            "theta": [],
            "alpha": [],
            "beta": [],
            "gamma": []
            }
        }
    }    

In [ ]:
# Load the epochs
mddEpochs = mne.read_epochs(fifFileMddEpochPath)
hcEpochs = mne.read_epochs(fifFileHcEpochPath)

print(len(mddEpochs))
print(len(hcEpochs))

print(mddEpochs.get_data()[0].shape)
print(hcEpochs.get_data()[0].shape)

In [ ]:
psdObjMdd = EEGPowerSpectrum(mddEpochs, 250, ifNormalize=True)
mddPsdFeatures = psdObjMdd.run()
psdObjCtrl = EEGPowerSpectrum(hcEpochs, 250, ifNormalize=True)
hcPsdFeatures = psdObjCtrl.run()
print(mddPsdFeatures.shape)
print(hcPsdFeatures.shape)

In [ ]:
# save at allFeatureDirPath + /psd
np.save(allFeatureDirPath + "/psd/mddPsdFeatures.npy", mddPsdFeatures)
np.save(allFeatureDirPath + "/psd/hcPsdFeatures.npy", hcPsdFeatures)

### 2. Phase Connectivity

In [ ]:
import dotenv
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
from brainmdd.features.phaseconnectivity import EEGPhaseConnectivity
import mne
import enum

dotenv.load_dotenv()

class ChannelMapping(enum.Enum):
    Fp1 = 22
    Fp2 = 9
    F3 = 24
    F4 = 124
    C3 = 36
    C4 = 104
    P3 = 52
    P4 = 92
    O1 = 70
    O2 = 83
    F7 = 33
    F8 = 122
    T3 = 45
    T4 = 108
    T5 = 58
    T6 = 96

# Declare Global Variables
NUM_CHANNELS = 16
MODMA_SFREQ = 250


# Declare Feature Dictionary
featureDict = {
    "phase_connectivity":{
        "mdd": [],
        "hc": []
    }
}

In [7]:
# Load the epochs
mddEpochs = mne.read_epochs(fifFileMddEpochPath)
hcEpochs = mne.read_epochs(fifFileHcEpochPath)
mddEpochs = mddEpochs.get_data()
controlEpochs = hcEpochs.get_data()
print(mddEpochs.shape)
print(controlEpochs.shape)

Reading /Users/wachiii/Workschii/brain-mdd/data/balanced16preprocess/mdd1624preprocessed.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...    9996.00 ms
        0 CTF compensation matrices available
Not setting metadata
713 matching events found
No baseline correction applied
0 projection items activated
Reading /Users/wachiii/Workschii/brain-mdd/data/balanced16preprocess/hc1624preprocessed.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...    9996.00 ms
        0 CTF compensation matrices available


/var/folders/kf/plh_bl0s64g1swyqvnc651pr0000gn/T/ipykernel_16757/3262787207.py:2: RuntimeWarning: This filename (/Users/wachiii/Workschii/brain-mdd/data/balanced16preprocess/mdd1624preprocessed.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  mddEpochs = mne.read_epochs(fifFileMddEpochPath)
/var/folders/kf/plh_bl0s64g1swyqvnc651pr0000gn/T/ipykernel_16757/3262787207.py:3: RuntimeWarning: This filename (/Users/wachiii/Workschii/brain-mdd/data/balanced16preprocess/hc1624preprocessed.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  hcEpochs = mne.read_epochs(fifFileHcEpochPath)


Not setting metadata
713 matching events found
No baseline correction applied
0 projection items activated
(713, 16, 2500)
(713, 16, 2500)


In [16]:
print(hcEpochs.get_data().shape)

(713, 16, 2500)


In [ ]:
phaseConObjMdd = EEGPhaseConnectivity(mddEpochs.get_data(), MODMA_SFREQ, 713, 16, 2500)
mddPhaseConFeatures = phaseConObjMdd.run()
phaseConObjControl = EEGPhaseConnectivity(hcEpochs, MODMA_SFREQ, 713, 16, 2500)
hcPhaseConFeatures = phaseConObjControl.run()

In [ ]:
# save at allFeatureDirPath + /psd
np.save(allFeatureDirPath + "/phase/hcPhaseFeatures.npy", mddPhaseConFeatures)
np.save(allFeatureDirPath + "/phase/hcPhaseFeatures.npy", hcPsdFeatures)

### 3. HFD

In [ ]:
import dotenv
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
from brainmdd.features.hfd import EEGHiguchiFractalDimension
import mne
import enum

dotenv.load_dotenv()

class ChannelMapping(enum.Enum):
    Fp1 = 22
    Fp2 = 9
    F3 = 24
    F4 = 124
    C3 = 36
    C4 = 104
    P3 = 52
    P4 = 92
    O1 = 70
    O2 = 83
    F7 = 33
    F8 = 122
    T3 = 45
    T4 = 108
    T5 = 58
    T6 = 96

# Declare Global Variables
NUM_CHANNELS = 16
MODMA_SFREQ = 250


# Declare Feature Dictionary
featureDict = {
    "HFD": {
        "mdd": [],
        "control": []
    }
}

In [22]:
# Load the epochs
mddEpochs = mne.read_epochs(fifFileMddEpochPath)
hcEpochs = mne.read_epochs(fifFileHcEpochPath)
mddEpochs = mddEpochs.get_data()
hcEpochs = hcEpochs.get_data()
print(mddEpochs.shape)
print(hcEpochs.shape)

Reading /Users/wachiii/Workschii/brain-mdd/data/balanced16preprocess/mdd1624preprocessed.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...    9996.00 ms
        0 CTF compensation matrices available


/var/folders/kf/plh_bl0s64g1swyqvnc651pr0000gn/T/ipykernel_16757/1695134047.py:2: RuntimeWarning: This filename (/Users/wachiii/Workschii/brain-mdd/data/balanced16preprocess/mdd1624preprocessed.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  mddEpochs = mne.read_epochs(fifFileMddEpochPath)


Not setting metadata
713 matching events found
No baseline correction applied
0 projection items activated
Reading /Users/wachiii/Workschii/brain-mdd/data/balanced16preprocess/hc1624preprocessed.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...    9996.00 ms
        0 CTF compensation matrices available
Not setting metadata
713 matching events found
No baseline correction applied
0 projection items activated


/var/folders/kf/plh_bl0s64g1swyqvnc651pr0000gn/T/ipykernel_16757/1695134047.py:3: RuntimeWarning: This filename (/Users/wachiii/Workschii/brain-mdd/data/balanced16preprocess/hc1624preprocessed.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  hcEpochs = mne.read_epochs(fifFileHcEpochPath)


(713, 16, 2500)
(713, 16, 2500)


In [26]:
hfdObjMdd = EEGHiguchiFractalDimension(mddEpochs)
mddHfdFeatures = hfdObjMdd.run()
hfdObjControl = EEGHiguchiFractalDimension(hcEpochs)
hcHfdFeatures = hfdObjControl.run()

print(mddHfdFeatures.shape)
print(hcHfdFeatures.shape)

(713, 16)
(713, 16)


In [28]:
# save at allFeatureDirPath + /psd
np.save(allFeatureDirPath + "/hfd/mddHfdFeatures.npy", mddHfdFeatures)
np.save(allFeatureDirPath + "/hfd/hcHfdFeatures.npy", hcHfdFeatures)

### 4. Coherences

In [29]:
import dotenv
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
from brainmdd.features.coherence import EEGCoherence
import mne
import enum

dotenv.load_dotenv()

class ChannelMapping(enum.Enum):
    Fp1 = 22
    Fp2 = 9 
    F3 = 24
    F4 = 124
    C3 = 36
    C4 = 104
    P3 = 52
    P4 = 92
    O1 = 70
    O2 = 83
    F7 = 33
    F8 = 122
    T3 = 45
    T4 = 108
    T5 = 58
    T6 = 96

# Declare Global Variables
NUM_CHANNELS = 16
MODMA_SFREQ = 250


# Declare Feature Dictionary
featureDict = {
    "coherence": {
        "mdd": {
            "delta": [],
            "theta": [],
            "alpha": [],
            "beta": [],
            "gamma": []
            },
        "control": {
            "delta": [],
            "theta": [],
            "alpha": [],
            "beta": [],
            "gamma": []
            }
        }
    } 


In [30]:
mddEpochs = mne.read_epochs(fifFileMddEpochPath)
hcEpochs = mne.read_epochs(fifFileHcEpochPath)
mddEpochs = mddEpochs.get_data()
hcEpochs = hcEpochs.get_data()
print(mddEpochs.shape)
print(hcEpochs.shape)

Reading /Users/wachiii/Workschii/brain-mdd/data/balanced16preprocess/mdd1624preprocessed.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...    9996.00 ms
        0 CTF compensation matrices available


/var/folders/kf/plh_bl0s64g1swyqvnc651pr0000gn/T/ipykernel_16757/1994406543.py:1: RuntimeWarning: This filename (/Users/wachiii/Workschii/brain-mdd/data/balanced16preprocess/mdd1624preprocessed.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  mddEpochs = mne.read_epochs(fifFileMddEpochPath)


Not setting metadata
713 matching events found
No baseline correction applied
0 projection items activated
Reading /Users/wachiii/Workschii/brain-mdd/data/balanced16preprocess/hc1624preprocessed.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...    9996.00 ms
        0 CTF compensation matrices available
Not setting metadata
713 matching events found
No baseline correction applied
0 projection items activated


/var/folders/kf/plh_bl0s64g1swyqvnc651pr0000gn/T/ipykernel_16757/1994406543.py:2: RuntimeWarning: This filename (/Users/wachiii/Workschii/brain-mdd/data/balanced16preprocess/hc1624preprocessed.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  hcEpochs = mne.read_epochs(fifFileHcEpochPath)


(713, 16, 2500)
(713, 16, 2500)


In [31]:
coherenceObjMdd = EEGCoherence(mddEpochs, MODMA_SFREQ)
mddCohFeatures = coherenceObjMdd.run()
coherenceObjControl = EEGCoherence(hcEpochs, MODMA_SFREQ)
hcCohFeatures = coherenceObjControl.run()

print(mddCohFeatures.shape)
print(hcCohFeatures.shape)

(713, 5, 120)
(713, 5, 120)


In [32]:
# save at allFeatureDirPath + /psd
np.save(allFeatureDirPath + "/coh/mddCohFeatures.npy", mddCohFeatures)
np.save(allFeatureDirPath + "/coh/hcCohFeatures.npy", hcCohFeatures)

### 5. Cd

In [34]:
import dotenv
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
from brainmdd.features.correlationdimension import EEGCorrelationDimension
import mne
import enum

dotenv.load_dotenv()

class ChannelMapping(enum.Enum):
    Fp1 = 22
    Fp2 = 9
    F3 = 24
    F4 = 124
    C3 = 36
    C4 = 104
    P3 = 52
    P4 = 92
    O1 = 70
    O2 = 83
    F7 = 33
    F8 = 122
    T3 = 45
    T4 = 108
    T5 = 58
    T6 = 96

# Declare Global Variables
NUM_CHANNELS = 16
MODMA_SFREQ = 250


# Declare Feature Dictionary
featureDict = {
    "CD": {
        "mdd": [],
        "control": []
    }
}

In [35]:
mddEpochs = mne.read_epochs(fifFileMddEpochPath)
hcEpochs = mne.read_epochs(fifFileHcEpochPath)
mddEpochs = mddEpochs.get_data()
hcEpochs = hcEpochs.get_data()
print(mddEpochs.shape)
print(hcEpochs.shape)

Reading /Users/wachiii/Workschii/brain-mdd/data/balanced16preprocess/mdd1624preprocessed.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...    9996.00 ms
        0 CTF compensation matrices available


/var/folders/kf/plh_bl0s64g1swyqvnc651pr0000gn/T/ipykernel_16757/1994406543.py:1: RuntimeWarning: This filename (/Users/wachiii/Workschii/brain-mdd/data/balanced16preprocess/mdd1624preprocessed.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  mddEpochs = mne.read_epochs(fifFileMddEpochPath)


Not setting metadata
713 matching events found
No baseline correction applied
0 projection items activated
Reading /Users/wachiii/Workschii/brain-mdd/data/balanced16preprocess/hc1624preprocessed.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...    9996.00 ms
        0 CTF compensation matrices available


/var/folders/kf/plh_bl0s64g1swyqvnc651pr0000gn/T/ipykernel_16757/1994406543.py:2: RuntimeWarning: This filename (/Users/wachiii/Workschii/brain-mdd/data/balanced16preprocess/hc1624preprocessed.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  hcEpochs = mne.read_epochs(fifFileHcEpochPath)


Not setting metadata
713 matching events found
No baseline correction applied
0 projection items activated
(713, 16, 2500)
(713, 16, 2500)


In [36]:
cdObjMdd = EEGCorrelationDimension(mddEpochs)
mddCdFeatures = cdObjMdd.run()
cdObjControl = EEGCorrelationDimension(hcEpochs)
hcCdFeatures = cdObjControl.run()

for epoch in mddCdFeatures:
    featureDict["CD"]["mdd"].append(epoch)
for epoch in hcCdFeatures:
    featureDict["CD"]["control"].append(epoch)

In [37]:
# save at allFeatureDirPath + /psd
np.save(allFeatureDirPath + "/cd/mddCdFeatures.npy", mddCdFeatures)
np.save(allFeatureDirPath + "/cd/hcCdFeatures.npy", hcCdFeatures)